# Assignment Module 2: Pet Classification

The goal of this assignment is to implement a neural network that classifies images of 37 breeds of cats and dogs from the [Oxford-IIIT-Pet dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/). The assignment is divided into two parts: first, you will be asked to implement from scratch your own neural network for image classification; then, you will fine-tune a pretrained network provided by PyTorch.

In [1]:
!git clone https://github.com/CVLAB-Unibo/ipcv-assignment-2.git

Cloning into 'ipcv-assignment-2'...
remote: Enumerating objects: 7371, done.
remote: Total 7371 (delta 0), reused 0 (delta 0), pack-reused 7371 (from 1)
Receiving objects: 100% (7371/7371), 753.77 MiB | 32.78 MiB/s, done.
Updating files: 100% (7396/7396), done.


In [2]:
import torch
from torch import nn, Tensor
import numpy as np
import random
from torchvision import models
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.transforms import v2 as transforms_v2
from torchvision.transforms import ToTensor
from torchvision.io import decode_image
import torch.optim.lr_scheduler as lr_scheduler
import os
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict, Optional
import time
from PIL import Image
from datetime import datetime
from torchvision.models import resnet18, ResNet18_Weights

In [3]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"Using {device} device")

Using cuda device


In [4]:
def fix_random(seed: int) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

fix_random(42)

In [5]:
class OxfordPetDataset(Dataset):
    def __init__(self, split: str, transform=None) -> None:
        super().__init__()
        self.root = Path("ipcv-assignment-2") / "dataset"
        self.split = split
        self.transform = transform
        self.names, self.labels = self._get_names_and_labels()
        tot_size_bytes = 0
        n_images = 0
        loaded_data = []
        idx = 0
        while idx < len(self.names):
            img_tensor = self.get_img_from_filesystem(idx)
            tot_size_bytes += img_tensor.numel() * img_tensor.element_size()
            n_images += 1
            loaded_data.append(img_tensor)
            idx += 1
        print("tot size", tot_size_bytes, "bytes, for", n_images, "images")
        self.data_tensor = torch.stack(loaded_data).to(device=device)
        self.labels = torch.Tensor(self.labels).type(torch.LongTensor).to(device=device)

    def __len__(self) -> int:
        return len(self.labels)

    def __getitem__(self, idx) -> Tuple[Tensor, int]:

        label = self.labels[idx]
        img = self.data_tensor[idx]
        img = self.transform(img).to(device)

        return img, label

    def get_img_from_filesystem(self, idx) -> Tensor:
        img_path = self.root / "images" / f"{self.names[idx]}.jpg"
        img = Image.open(img_path).convert("RGB")
        img = transforms_v2.Resize((256,256))(img)
        img = transforms_v2.ToTensor()(img)
        return img


    def get_num_classes(self) -> int:
        return max(self.labels) + 1

    def _get_names_and_labels(self) -> Tuple[List[str], List[int]]:
        names = []
        labels = []

        with open(self.root / "annotations" / f"{self.split}.txt") as f:
            for line in f:
                if(line[0] == "#"):
                    continue
                name, label = line.replace("\n", "").split(" ")
                names.append(name),
                labels.append(int(label) - 1)

        return names, labels

In [6]:
def get_model_resnet18():
    # Define the model and local path for saving weights
    model_resnet_18 = models.resnet18()
    weights_path = "./resnet18_weights.pth"

    # Check if the file exists locally
    if os.path.exists(weights_path):
        print("✅ Loading existing ResNet-18 weights...")
        state_dict = torch.load(weights_path, map_location="cpu")
        model_resnet_18.load_state_dict(state_dict)
    else:
        print("⬇️  Downloading ResNet-18 weights...")
        model_resnet_18 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)  # Downloads pretrained weights
        torch.save(model_resnet_18.state_dict(), weights_path)
        print("💾 Weights saved locally at:", weights_path)
    return model_resnet_18

In [7]:
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

## Part 1: design your own network

Your goal is to implement a convolutional neural network for image classification and train it from scratch on `OxfordPetDataset`. You should consider yourselves satisfied once you obtain a classification accuracy on the test split of ~60%. You are free to achieve this however you want, except for a few rules you must follow:

- Compile this notebook by displaying the results obtained by the best model you found throughout your experimentation; then show how, by removing some of its components, its performance drops. In other words, do an *ablation study* to prove that your design choices have a positive impact on the final result.

- Do not instantiate an off-the-self PyTorch network. Instead, construct your network as a composition of existing PyTorch layers. In more concrete terms, you can use e.g. `torch.nn.Linear`, but you cannot use e.g. `torchvision.models.alexnet`.

- Show your results and ablations with plots, tables, images, etc. — the clearer, the better.

Don't be too concerned with your model performance: the ~60% is just to give you an idea of when to stop. Keep in mind that a thoroughly justified model with lower accuracy will be rewarded more points than a poorly experimentally validated model with higher accuracy.

In [8]:

# Defining custom Dense Block
class DenseBlock(nn.Module):
    def __init__(self, num_convs:int, growth_rate:int, has_transition:bool = True):
        super(DenseBlock, self).__init__()
        self.num_convs = num_convs
        self.growth_rate = growth_rate
        layer = []
        for i in range(num_convs):
            layer.append(self.conv_block(growth_rate))
        self.dense_net = nn.Sequential(*layer)
        self.transition_net = self.transition_block() if has_transition else None

    def conv_block(self, num_channels):
        return (
            nn.Sequential(
                nn.LazyBatchNorm2d(),
                nn.ReLU(),
                nn.LazyConv2d(num_channels, 3, padding=1))
        )

    def transition_block(self): #transition block to reduce channel dimensionality (conv has to be 1x1)
        compression_factor = 0.5
        num_transition_channels = int(compression_factor * (self.growth_rate * self.num_convs))
        return (
            nn.Sequential(
                nn.LazyBatchNorm2d(),
                nn.ReLU(),
                nn.LazyConv2d(num_transition_channels, 1),
                nn.AvgPool2d(2, stride=2)
                )
        )

    def forward(self, X):
        # In the Dense block, every layer has in input the concatenation of all the previous ones
        for block in self.dense_net:
            last_out = block(X)
            X = torch.cat((X, last_out), dim=1)
        if self.transition_net != None:
            X = self.transition_net(X)
        return X

In [9]:

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self, num_dense_blocks, num_conv_each_dense, growth_rate, dropout):
        super().__init__()
        stem_block = [ #initial stem block, like resnet
            nn.LazyConv2d(32, 7, stride=2),
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.MaxPool2d(3, stride=2),
        ]

        dense_core = [
            DenseBlock(num_convs=num_conv_each_dense[i], growth_rate=growth_rate, has_transition= True ) for i in range(num_dense_blocks)
        ]

        classification_block = [
            nn.LazyBatchNorm2d(),
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Dropout(dropout),
            nn.LazyLinear(37)
        ]

        self.net_stack = nn.Sequential(*stem_block, *dense_core, *classification_block)

    def forward(self, x):
        result = self.net_stack(x)
        return result

In [10]:
def train(dataloader, model, loss_fn, optimizer, scheduler, lr_data):
    model.train()
    total_loss = 0.0
    num_batches = 0

    for X, y in dataloader:
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        num_batches += 1
        lr_data.append(optimizer.param_groups[0]["lr"])

    avg_loss = total_loss / num_batches
    return avg_loss

In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    accuracy = 100*correct
    return accuracy, test_loss

In [12]:

def evaluate_model(test_loader, model):
    model.eval()
    correct = 0
    total = 0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs, labels
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.append(preds)
            all_labels.append(labels)

    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    # --- Accuracy ---
    accuracy = correct / total

    # --- Confusion Matrix ---
    num_classes = outputs.shape[1]
    conf_matrix = torch.zeros(num_classes, num_classes, dtype=torch.int64)
    for t, p in zip(all_labels, all_preds):
        conf_matrix[t, p] += 1

    # --- Precision, Recall, F1 ---
    tp = conf_matrix.diag()
    fp = conf_matrix.sum(dim=0) - tp
    fn = conf_matrix.sum(dim=1) - tp

    precision = tp / (tp + fp + 1e-12)
    recall = tp / (tp + fn + 1e-12)
    f1 = 2 * precision * recall / (precision + recall + 1e-12)

    # Weighted (by class support)
    support = conf_matrix.sum(dim=1)
    weighted_precision = (precision * support).sum() / support.sum()
    weighted_recall = (recall * support).sum() / support.sum()
    weighted_f1 = (f1 * support).sum() / support.sum()

    return {
        "test_accuracy": accuracy,
        "test_precision": weighted_precision.item(),
        "test_recall": weighted_recall.item(),
        "test_f1": weighted_f1.item()
    }

In [ ]:
## declaration of data augumentation stacks
training_data_augumentation_stacks = {
    "default" : transforms_v2.Compose([
        transforms_v2.Resize((256,256)),
        transforms_v2.RandomResizedCrop(size=(224, 224)),
        transforms_v2.RandomHorizontalFlip(p=0.5),
        transforms_v2.ColorJitter(
            brightness=0.5,
            contrast=0.5,
            saturation=0.5,
            hue=0.1
        ),
        transforms_v2.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 1.0)),
        transforms_v2.ToDtype(torch.float32, scale=True),
        transforms_v2.Normalize(mean=MEAN,
                    std=STD),
    ]),

    "same_as_default_but_lighter" : transforms_v2.Compose([
        transforms_v2.Resize((256,256)),
        transforms_v2.RandomResizedCrop(size=(224, 224)),
        transforms_v2.RandomHorizontalFlip(p=0.5),
        transforms_v2.ColorJitter(
            brightness=0.1,
            contrast=0.1,
            saturation=0.1,
            hue=0.05
        ),
        transforms_v2.GaussianBlur(kernel_size=(3, 3), sigma=(0.2)),
        transforms_v2.ToDtype(torch.float32, scale=True),
        transforms_v2.Normalize(mean=MEAN,
                    std=STD),
    ]),
"trivial_augument_wide": transforms_v2.Compose([
    transforms_v2.Resize((256,256)),
    transforms_v2.TrivialAugmentWide(),
    transforms_v2.RandomResizedCrop(size=(224, 224)),
    transforms_v2.ToDtype(torch.float32, scale=True),
    transforms_v2.Normalize(MEAN, STD),
])
}


In [14]:
def run_experiment(parameters):
    BATCH_SIZE = parameters.get("batch_size", 64)
    N_DENS_BLKS = parameters.get("n_dense_blocks", 3)
    N_CONVS_EACH_BLCK = parameters.get("n_conv_each_block", [4,8,7])
    GRWTH_RATE = parameters.get("growth_rate", 12)
    DROPOUT = parameters.get("dropout", 0.1)
    BASE_LR = parameters.get("base_learning_rate", 0.001)
    WARMUP_ITERS = parameters.get("warmup_iter", 7500)
    MAIN_SCH_KICK = parameters.get("main_sched_epochs_kick_in", 60)
    EPOCHS = parameters.get("n_epochs", 100)
    WEIGHT_DECAY = parameters.get("weight_decay", 0.001)
    TRAINING_TRANSFORM_STACK = training_data_augumentation_stacks[parameters.get("transform_stack_name", "default")]
    EXPERIMENT_TYPE = parameters.get("experiment_type", "default")

    validation_transform_stack = transforms_v2.Compose([
        transforms_v2.Resize(256),
        transforms_v2.CenterCrop(224),
        transforms_v2.Normalize(MEAN, STD)
    ])

    train_dataset = OxfordPetDataset(split="train" , transform=TRAINING_TRANSFORM_STACK)
    validation_dataset = OxfordPetDataset(split="val", transform=validation_transform_stack)
    test_dataset = OxfordPetDataset(split="test", transform=validation_transform_stack)

    # Create data loaders.
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

    if EXPERIMENT_TYPE == "default" or EXPERIMENT_TYPE == "dense_net":
        model = NeuralNetwork(num_dense_blocks=N_DENS_BLKS, num_conv_each_dense=N_CONVS_EACH_BLCK, growth_rate=GRWTH_RATE, dropout=DROPOUT).to(device)
    elif EXPERIMENT_TYPE == "resnet18":
        model_resnet_18 = get_model_resnet18()
        num_features = model_resnet_18.fc.in_features
        model_resnet_18.fc = nn.Sequential(
            nn.Dropout(DROPOUT),
            nn.Linear(num_features, 37)
        )
        # Freeze everything
        for param in model_resnet_18.parameters():
            param.requires_grad = False
        # Unfreeze only the head
        for param in model_resnet_18.fc.parameters():
            param.requires_grad = True
        model = model_resnet_18.to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=BASE_LR, weight_decay=WEIGHT_DECAY)

    warmup_scheduler = lr_scheduler.LinearLR(optimizer, start_factor=0.01, end_factor=1., total_iters=WARMUP_ITERS)
    main_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.01, patience=8)

    batch_lrs = []           # record learning rate per batch
    epoch_end_indices = []   # store index (batch count) at the end of each epoch

    net_performance_data = {
        "train_loss": [],
        "val_loss": [],
        "val_accuracy": []
    }

    t1 = time.time()

    for t in range(EPOCHS):
        net_performance_data["train_loss"].append( train(train_dataloader, model, loss_fn, optimizer, warmup_scheduler, batch_lrs) )
        val_accuracy, val_loss = test(validation_dataloader, model, loss_fn)
        if t > MAIN_SCH_KICK: main_scheduler.step(val_accuracy)
        net_performance_data["val_loss"].append( val_loss )
        net_performance_data["val_accuracy"].append( val_accuracy )
        epoch_end_indices.append(len(batch_lrs))

    t2 = time.time()
    elapsed_time = t2-t1

    print("Done in ", elapsed_time, " sec")

    print("Best accuracy ", np.max(net_performance_data["val_accuracy"]))

    net_performance_data_df = pd.DataFrame(data=net_performance_data)

    #add all experiment parameters to the result dataframe
    for key in parameters:
        if isinstance(parameters[key], list):
            for i, el in enumerate(parameters[key]):
                net_performance_data_df[key + "_"+ str(i)] = el
        else:
            net_performance_data_df[key] = parameters[key]

    # add elapsed time
    net_performance_data_df["elapsed_time_sec"] = elapsed_time

    # create a column for epochs steps indexes
    net_performance_data_df = net_performance_data_df.reset_index().rename(columns={"index":"epoch"})

    # add final evaluation stats
    eval_performaces = evaluate_model(test_dataloader, model)
    for metric in eval_performaces:
        net_performance_data_df[metric] = eval_performaces[metric]

    return net_performance_data_df


In [ ]:
parameters_base_deactivated = [
{
    "experiment_name": "24_grwth_rate_baseline",
    "batch_size" : 64,
    "n_dense_blocks": 3,
    "n_conv_each_block": [6,16,14],
    "growth_rate": 24,
    "dropout": 0.2,
    "base_learning_rate": 0.001,
    "warmup_iter": 7500,
    "main_sched_epochs_kick_in": 70,
    "n_epochs": 180,
    "weight_decay": 0.001,
    "transform_stack_name": "default"
},
{
    "experiment_name": "high_dropout",
    "batch_size" : 64,
    "n_dense_blocks": 3,
    "n_conv_each_block": [6,16,14],
    "growth_rate": 24,
    "dropout": 0.4,
    "base_learning_rate": 0.001,
    "warmup_iter": 7500,
    "main_sched_epochs_kick_in": 70,
    "n_epochs": 180,
    "weight_decay": 0.001,
    "transform_stack_name": "default"
},
{
    "experiment_name": "capacity_increase_blocks_growth",
    "batch_size": 64,
    "n_dense_blocks": 4,
    "n_conv_each_block": [6, 12, 16, 14],
    "growth_rate": 32,
    "dropout": 0.2,
    "base_learning_rate": 0.001,
    "warmup_iter": 7500,
    "main_sched_epochs_kick_in": 70,
    "n_epochs": 180,
    "weight_decay": 0.001,
    "transform_stack_name": "default"
},
{
    "experiment_name": "lightweight_model_ablation",
    "batch_size": 96,
    "n_dense_blocks": 3,
    "n_conv_each_block": [4, 10, 10],
    "growth_rate": 16,
    "dropout": 0.2,
    "base_learning_rate": 0.001,
    "warmup_iter": 5000,
    "main_sched_epochs_kick_in": 60,
    "n_epochs": 180,
    "weight_decay": 0.0005,
    "transform_stack_name": "default"
},
{
    "experiment_name": "deep_dense_blocks",
    "batch_size": 32,
    "n_dense_blocks": 3,
    "n_conv_each_block": [12, 24, 24],
    "growth_rate": 24,
    "dropout": 0.2,
    "base_learning_rate": 0.0008,
    "warmup_iter": 10000,
    "main_sched_epochs_kick_in": 90,
    "n_epochs": 180,
    "weight_decay": 0.0015,
    "transform_stack_name": "default"
},
{
    "experiment_name": "weight_decay_heavy",
    "batch_size": 64,
    "n_dense_blocks": 3,
    "n_conv_each_block": [6,16,14],
    "growth_rate": 24,
    "dropout": 0.2,
    "base_learning_rate": 0.001,
    "warmup_iter": 7500,
    "main_sched_epochs_kick_in": 70,
    "n_epochs": 180,
    "weight_decay": 0.007,
    "transform_stack_name": "default"
},
{
    "experiment_name": "near_zero_weight_decay",
    "batch_size": 64,
    "n_dense_blocks": 3,
    "n_conv_each_block": [6,16,14],
    "growth_rate": 24,
    "dropout": 0.2,
    "base_learning_rate": 0.001,
    "warmup_iter": 7500,
    "main_sched_epochs_kick_in": 70,
    "n_epochs": 180,
    "weight_decay": 0.00005,
    "transform_stack_name": "default"
},
{
    "experiment_name": "large_warmup",
    "batch_size": 64,
    "n_dense_blocks": 3,
    "n_conv_each_block": [6,16,14],
    "growth_rate": 24,
    "dropout": 0.2,
    "base_learning_rate": 0.0005,
    "warmup_iter": 20000,
    "main_sched_epochs_kick_in": 90,
    "n_epochs": 180,
    "weight_decay": 0.001,
    "transform_stack_name": "default"
},
{
    "experiment_name": "batch32_lr_double",
    "batch_size": 32,
    "n_dense_blocks": 3,
    "n_conv_each_block": [6,16,14],
    "growth_rate": 24,
    "dropout": 0.2,
    "base_learning_rate": 0.002,
    "warmup_iter": 7500,
    "main_sched_epochs_kick_in": 70,
    "n_epochs": 180,
    "weight_decay": 0.001,
    "transform_stack_name": "default"
},
{
    "experiment_name": "big_128batch",
    "batch_size": 128,
    "n_dense_blocks": 3,
    "n_conv_each_block": [6,16,14],
    "growth_rate": 24,
    "dropout": 0.2,
    "base_learning_rate": 0.001,
    "warmup_iter": 7500,
    "main_sched_epochs_kick_in": 70,
    "n_epochs": 180,
    "weight_decay": 0.001,
    "transform_stack_name": "default"
},
{
    "experiment_name": "light_data_aug",
    "batch_size": 64,
    "n_dense_blocks": 3,
    "n_conv_each_block": [6,16,14],
    "growth_rate": 24,
    "dropout": 0.2,
    "base_learning_rate": 0.001,
    "warmup_iter": 7500,
    "main_sched_epochs_kick_in": 70,
    "n_epochs": 180,
    "weight_decay": 0.001,
    "transform_stack_name": "same_as_default_but_lighter"
}
]
parameters_base = []

In [16]:
total_results = []

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

### Actual run of the experiments
for par in parameters_base:
    total_results.append(run_experiment(par))
    # update partial results after each experiment
    df_combined = pd.concat(total_results, ignore_index=True)
    df_combined.to_csv(f"./results_{timestamp}.csv")

/usr/local/lib/python3.12/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


tot size 2885419008 bytes, for 3669 images
tot size 1442316288 bytes, for 1834 images
tot size 1451753472 bytes, for 1846 images
Done in  258.4708516597748  sec
Best accuracy  17.230098146128682


/usr/local/lib/python3.12/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


tot size 2885419008 bytes, for 3669 images
tot size 1442316288 bytes, for 1834 images
tot size 1451753472 bytes, for 1846 images
Done in  26.26095986366272  sec
Best accuracy  6.9792802617230105


## Part 2: fine-tune an existing network

Your goal is to fine-tune a pretrained ResNet-18 model on `OxfordPetDataset`. Use the implementation provided by PyTorch, i.e. the opposite of part 1. Specifically, use the PyTorch ResNet-18 model pretrained on ImageNet-1K (V1). Divide your fine-tuning into two parts:

### 2A.
 First, fine-tune the ResNet-18 with the same training hyperparameters you used for your best model in part 1.


### 2B.
Then, tweak the training hyperparameters in order to increase the accuracy on the test split. Justify your choices by analyzing the training plots and/or citing sources that guided you in your decisions — papers, blog posts, YouTube videos, or whatever else you may find useful. You should consider yourselves satisfied once you obtain a classification accuracy on the test split of ~90%.

In [ ]:
parameters_finetune_base = [
{
    "experiment_name": "delete_it",
    "batch_size": 64,
    "dropout": 0.2,
    "base_learning_rate": 0.001,
    "warmup_iter": 1500,
    "main_sched_epochs_kick_in": 30,
    "n_epochs": 2,
    "weight_decay": 0.001,
    "transform_stack_name": "default"
},
{
    "experiment_name": "same_as_densenet",
    "batch_size": 64,
    "dropout": 0.2,
    "base_learning_rate": 0.001,
    "warmup_iter": 1500,
    "main_sched_epochs_kick_in": 30,
    "n_epochs": 70,
    "weight_decay": 0.001,
    "transform_stack_name": "default"
},
{
    "experiment_name": "better_model",
    "batch_size": 64,
    "dropout": 0.2,
    "base_learning_rate": 0.0001,
    "warmup_iter": 1500,
    "main_sched_epochs_kick_in": 30,
    "n_epochs": 70,
    "weight_decay": 0.001,
    "transform_stack_name": "trivial_augument_wide"
},
{
    "experiment_name": "better_model_smaller_batches",
    "batch_size": 32,
    "dropout": 0.2,
    "base_learning_rate": 0.0001,
    "warmup_iter": 1500,
    "main_sched_epochs_kick_in": 30,
    "n_epochs": 70,
    "weight_decay": 0.001,
    "transform_stack_name": "trivial_augument_wide"
},
{
    "experiment_name": "better_model_larger_batches",
    "batch_size": 128,
    "dropout": 0.2,
    "base_learning_rate": 0.0001,
    "warmup_iter": 1500,
    "main_sched_epochs_kick_in": 30,
    "n_epochs": 70,
    "weight_decay": 0.001,
    "transform_stack_name": "trivial_augument_wide"
},
{
    "experiment_name": "better_model_high_dropout",
    "batch_size": 64,
    "dropout": 0.4,
    "base_learning_rate": 0.0001,
    "warmup_iter": 1500,
    "main_sched_epochs_kick_in": 30,
    "n_epochs": 70,
    "weight_decay": 0.001,
    "transform_stack_name": "trivial_augument_wide"
},
]

In [ ]:
total_results = []

### Actual run of the experiments
for par in parameters_finetune_base:
    total_results.append(run_experiment(par))
    # update partial results after each experiment
    df_combined = pd.concat(total_results, ignore_index=True)
    df_combined.to_csv(f"./results_finetune_{timestamp}.csv")

/usr/local/lib/python3.12/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
